In [2]:
import cv2
import time
import torch
import numpy as np
from customMTCNN import CustomMTCNN  # Your MTCNN model
from faceEmbedd import FaceEmbedder  # Your face embedding model
from hybrid import HybridFaceRecognition  # Hybrid tracking system

# Load models
mtcnn = CustomMTCNN()
embedder = FaceEmbedder()
hybrid_system = HybridFaceRecognition(mtcnn, embedder)

# Open a video source (0 = webcam)
cap = cv2.VideoCapture("test_video.mp4")  # Replace with your video file or 0 for webcam

# Track performance
start_time = time.time()
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    frame_resized = cv2.resize(frame, (640, 480))

    ## 1️⃣ MTCNN Face Detection Only
    start = time.time()
    img_tensor = torch.from_numpy(frame_resized).permute(2, 0, 1).float().unsqueeze(0) / 255.0
    boxes, _ = mtcnn.detect(img_tensor)
    mtcnn_time = time.time() - start

    for (x1, y1, x2, y2) in boxes:
        cv2.rectangle(frame_resized, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 255), 2)

    ## 2️⃣ Face Embedding Model
    start = time.time()
    for (x1, y1, x2, y2) in boxes:
        face_crop = frame_resized[int(y1):int(y2), int(x1):int(x2)]
        face_tensor = torch.from_numpy(face_crop).permute(2, 0, 1).float() / 255.0
        embedding = embedder.get_embedding(face_tensor)
    embedding_time = time.time() - start

    ## 3️⃣ Hybrid Approach: Detection + Tracking + Caching
    start = time.time()
    detected_faces = hybrid_system.detect_and_track(frame_resized)
    hybrid_time = time.time() - start

    for face in detected_faces:
        x, y, w, h = [int(coord) for coord in face['bbox']]
        cv2.rectangle(frame_resized, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame_resized, f"Face {face['id']}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display Results
    cv2.imshow("Face Detection Comparison", frame_resized)

    print(f"Frame {frame_count} | MTCNN Time: {mtcnn_time:.3f}s | Embedding Time: {embedding_time:.3f}s | Hybrid Time: {hybrid_time:.3f}s")

    # Exit on 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Compute FPS
end_time = time.time()
fps = frame_count / (end_time - start_time)
print(f"Overall FPS: {fps:.2f}")

cap.release()
cv2.destroyAllWindows()


Overall FPS: 0.00
